In [6]:
# # 노트북에서 실행
# from pathlib import Path

# # code 폴더에 __init__.py 생성
# code_init = Path.cwd().parent / 'code' / '__init__.py'
# code_init.touch()

# print(f"✓ {code_init} 생성 완료")

In [7]:
from typing import Tuple, List
import polars as pl
from pprint import pprint
import sys
from pathlib import Path
import psutil
import pandas as pd
import time
import numpy as np

In [ ]:
import sys
from pathlib import Path

# 상대 경로 사용
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'

# 맨 앞에 추가
sys.path.insert(0, str(PROJECT_ROOT))

# Python 내장 code 모듈 캐시만 임시 제거
if 'code' in sys.modules:
    del sys.modules['code']

# 이제 import
#from code.loading import DataLoader
from src.utils.statistic import Chi2Test

In [9]:
import polars as pl
import pandas as pd

In [10]:
df = pl.read_csv(
    "maude_extracted_sample2.csv",
    infer_schema_length=0
) # 일단 모든 data 문자열로 받음

In [11]:
# DataFrame을 LazyFrame으로 변환
lf = df.lazy()

In [12]:
df = lf.clone()

In [13]:
total_rows = df.select(pl.len()).collect().item()
total_cols = len(df.collect_schema().names())
print(f"전체 행: {total_rows:,}개, 전체 컬럼: {total_cols}개")

전체 행: 1,000개, 전체 컬럼: 409개


## 중복 제거 (전처리 코드로 돌리게 된다면 삭제)

In [14]:
# dedup_cols = [
#     'report_number',
#     'date_of_event', 
#     'device_0_manufacturer_d_name',
#     'device_0_udi_di',
#     'device_0_lot_number',
#     'device_0_udi_public'
# ]

# # Unknown / N/A 패턴 리스트
# na_patterns = r'^None$|^UNK|NOT APPLICABLE|NOT REPORTED|^N/A$|^NA$|^$|\s+$|^UNKNOWN$|^NI$|^NULL$'

In [15]:
# # 일단 중복된 행들만 확인
# # 조합 (6개 컬럼)의 개수
# df_with_cnt = df.with_columns(
#     pl.len().over(dedup_cols).alias('duplicate_cnt')
# )

# # cnt가 2 이상이 경우에만 
# # cnt가 1인 경우에는 삭제할 필요가 없으므로
# df_duplicates_only = df_with_cnt.filter(
#     pl.col('duplicate_cnt') >= 2
# )

# duplicate_cnt = df_duplicates_only.select(pl.len()).item()
# print(f"중복된 행의 개수: {duplicate_cnt:,}개")

# unique_cnt = df.unique(subset=dedup_cols, maintain_order=True).select(pl.len()).item()
# print(f"유일한 행의 개수: {unique_cnt:,}개")

In [16]:
# def remove_na_values(df, dedup_cols, na_patterns, verbose=True):
#     """
#     Na / Unknwon 값이 있는 행을 제거하는 함수
    
#     작동방식 :
#     1. 각 컬럼에 대해 유효한 값인지 체크
#     2. 모든 조건은 포함한(모두 만족하는) 행으로 필터링
#     3. 필터 적용

#     Parameters:
#     df : polars.DataFrame
#         원본 DF
#     dedup_cols : list
#         -> 모두 유효해야 함
#     na_patterns : str
#         NA / Unknown 패턴 정규식
#     verbose : bool
#         -> 현재 진행상황 출력해서 확인할 수 있게

#     Returns: 
#     polars.LazyFrame
#         비어진 값이나 모르는 값이 없는 DF
#     """

#     # 진행상확 확인
#     if verbose:
#         print("na 값 제거")
#         print(f"패턴: {na_patterns}")

#     # 제거 되기 전 개수 확인
#     before_cnt = df.select(pl.len()).item()
#     if verbose:
#         print(f"제거 전 행 개수 : {before_cnt:,}개")
    
#     # ===
#     # 각 컬럼별로 필터 조건
#     # ===
#     conditions = []

#     for col in dedup_cols:
#         # 컬럼이 존재하는지 확인
#         if col in df.collect_schema().names():
#             # 유효한 값의 조건
#             # null / na_patterns 패턴에 매칭되지 않는 값
#             cond = (
#                 pl.col(col).is_not_null()
#                 &
#                 ~ pl.col(col).cast(pl.Utf8).str.to_uppercase().str.contains(na_patterns)
#                 # ~ 은 not 연산자
#             )
#             conditions.append(cond)

#             if verbose:
#                 print(f"컬럼 '{col}'에 대해 na/unknown 값 제거 조건 추가")
#         else:
#             if verbose:
#                 print(f"컬럼 '{col}'이(가) 존재하지 않음. 건너뜀")
        
# # 예외처리
#     if not conditions:
#         if verbose:
#             print("제거할 조건이 없음. 원본 DF 반환")
#         return df

# # ===
# # 모든 조건을 and 조건으로 결합
# # (모든 조건을 만족해야 함)
# # ===
#     final_condition = conditions[0]
#     for cond in conditions[1:]:
#         final_condition = final_condition & cond

# # 필터 적용
#     df_cleaned = df.filter(final_condition)

# # 결과
#     after_cnt = df_cleaned.select(pl.len()).item()
#     removed_cnt = before_cnt - after_cnt

#     if verbose:
#         print(f"제거 후 행 개수 : {after_cnt:,}개")
#         print(f"제거된 행 개수 : {removed_cnt:,}개")

#         return df_cleaned

In [17]:
# def analyze_duplicates(df, group_cols, verbose = True):
#     """
#     중복 데이터 분석 함수들

#     작동 방식 
#     1. 전체 개수 확인
#     2. 고유(unique) 개수 확인
#     3. 전체 - 고유 = 중복 개수 확인

#     Parameters:
#     df : polars.DataFrame -> 원본
#     dedup_cols : list -> 중복 확인 컬럼
#     verbose : bool -> 진행상황 출력 여부

#     returns:

#     tuple : (전체 개수, 고유 개수, 중복 개수)
#     """

#     if verbose:
#         print(f" 중복 확인")

#     # 전체 개수
#     total_cnt = df.select(pl.len()).item()

#     # 고유 개수
#     unique_cnt = df.unique(
#         subset = group_cols, # 중복 ㅎ판단
#         maintain_order = True
#     ).select(pl.len()).item()

#     # 중복 개수
#     duplicate_cnt = total_cnt - unique_cnt

#     if verbose:
#         print(f"전체 개수 : {total_cnt:,}개")
#         print(f"고유 개수 : {unique_cnt:,}개")
#         print(f"중복 개수 : {duplicate_cnt:,}개")
#         for i, col in enumerate(dedup_cols, start=1):
#             print(f"{i}. {col}")

#     return total_cnt, unique_cnt, duplicate_cnt

In [18]:
# def remove_duplicates(df, dedup_cols, keep = 'first', verbose = True):
#     """
#     중복 데이터 제거 함수

#     작동 방식
#     1. dedup_cols 기준으로 중복 판단
#     2. keep 옵션에 따라 첫번째/마지막 행 유지
#     3. 중복 제거된 DF 반환

#     Parameters:
#     df : polars.DataFrame -> 원본 DF
#     dedup_cols : list -> 중복 판단 컬럼 리스트
#     keep : str -> 'first' or 'last'
#         'first' : 첫번째 행 유지
#         'last' : 마지막 행 유지
#     verbose : bool -> 진행상황 출력 여부

#     Returns:
#     polars.DataFrame -> 중복 제거된 DF
#     """

#     if verbose:
#         print("중복 제거 시작")
#         print(f"중복 판단 컬럼: {dedup_cols}")
#         print(f"유지 옵션: {keep}")

#     # 중복 제거
#     df_deduped = df.unique(
#         subset = dedup_cols,
#         maintain_order = True,
#         keep = 'first'
#     )

#     if verbose:
#         before_cnt = df.select(pl.len()).item()
#         after_cnt = df_deduped.select(pl.len()).item()
#         removed_cnt = before_cnt - after_cnt

#         print(f"제거 전 행 개수 : {before_cnt:,}개")
#         print(f"제거 후 행 개수 : {after_cnt:,}개")
#         print(f"제거된 행 개수 : {removed_cnt:,}개")

#     return df_deduped

In [19]:
# # 완전한 파이프라인
# df_all_cleaned = remove_na_values(df, dedup_cols, na_patterns)
# total, unique, dup = analyze_duplicates(df_all_cleaned, dedup_cols)
# df_all_final = remove_duplicates(df_all_cleaned, dedup_cols, keep='first')


## 클래스3에서 진행되는거겠죠? (전처리 된 파일의 코드를 뽑은 프롬프트로 실행하면 삭제)

In [20]:
# print("="*80)
# print("데이터 타입 통일 및 Class 3 필터링")
# print("="*80)

# # 1. device_class 컬럼 찾기
# device_class_cols = [
#     col for col in df.collect_schema().names()
#     if 'device_' in col and 'openfda_device_class' in col
# ]

# print(f"\n찾은 컬럼: {device_class_cols}")

# # 3. String으로 통일
# print(f"\nString 타입으로 통일 중...")
# df = df.with_columns([
#     pl.col(col).cast(pl.Utf8).alias(col) for col in device_class_cols
# ])

# print(f"\n변환 후 타입:")
# for col in device_class_cols:
#     dtype = df.select(pl.col(col)).collect_schema()[col]
#     print(f"  {col}: {dtype}")

In [21]:
# df_class3 = df.filter()
# row_count = df_class3.select(pl.len()).collect().item()
# print(f"{row_count:,}개")

In [22]:
# df
# = df_class3.clone()

In [23]:
# df_class3 = df.clone()  # 필요시 사용

# 제품코드별

## 1. 고장 유형 탐색 (EDA)

###  1-1. 이상 사례 발생 top 10 제품코드 / 결함 유형 발생 비율

In [24]:
# 함수화 
def get_top10_products(df, product_col = 'device_0_device_report_product_code'):
    """
    코드 중에서 이상 사례 발생 수가 높은 상위 10개의 제품코드 (class 3)
    
    Args : 
        df : DF
        product_col = 제품코드 컬럼명

    Returns : 
        top 10 결과 , 제품 코드 리스트 
    """
    print("=" * 50)
    print("top 10  제품코드 분석")
    print("=" * 50)

   # 전체 제품 코드에서 이상 사례 발생 수가 높은 상위 10개 제품 코드 확인
    top10_result = (
        df
        .group_by("device_0_device_report_product_code")
        .agg(pl.count().alias("total_count"))
        .sort("total_count", descending=True)
        .head(10)
        .collect()
    )
    
    print("top 10 제품코드")
    print(top10_result)

    # 상위 10개 제품 코드 리스트화
    top10_product_codes = top10_result['device_0_device_report_product_code'].to_list()
        # top10_result[product_col].to_list()

    return top10_result, top10_product_codes

In [25]:
top10_result, top10_product_codes = get_top10_products(df)

top 10  제품코드 분석
top 10 제품코드
shape: (10, 2)
┌─────────────────────────────────┬─────────────┐
│ device_0_device_report_product… ┆ total_count │
│ ---                             ┆ ---         │
│ str                             ┆ u32         │
╞═════════════════════════════════╪═════════════╡
│ OZP                             ┆ 228         │
│ LGW                             ┆ 55          │
│ PQF                             ┆ 50          │
│ FTR                             ┆ 49          │
│ LWS                             ┆ 48          │
│ DTB                             ┆ 41          │
│ OZD                             ┆ 37          │
│ NVN                             ┆ 37          │
│ OYC                             ┆ 33          │
│ MKJ                             ┆ 26          │
└─────────────────────────────────┴─────────────┘


/var/folders/qk/10b0q2j971j6d2py2x39ys200000gn/T/ipykernel_13931/624432124.py:21: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  .agg(pl.count().alias("total_count"))


In [26]:
def get_total_ratio(df):
    """
    전체 기기에서의 각 결함 유형 발생 비율

    Args :
        df : 분석할 DF
        defect_col : 분석할 컬럼 (결함 유형)

    Returns : 
    """
    print("=" * 50)
    print("기기 결함 유형")
    print("=" * 50)

    # 전체 기기 이상 사례 데이터에서 각 결함 유형의 발생 비율 확인
    ratio_result = (
        df
        .group_by("defect_type")  # 결함 유형별로 그룹화
        .agg(pl.len().alias("count"))
        .with_columns(
            (pl.col("count") / pl.col("count").sum() * 100)
            .round(2)
            .alias("percentage")
        )
        .sort("percentage", descending=True)
        .collect()  # LazyFrame이면 필요
    )

    print(ratio_result.to_pandas())

    return ratio_result

In [27]:
ratio_result = get_total_ratio(df)

기기 결함 유형
                    defect_type  count  percentage
0                         Other    294        29.4
1              Electrical/Power    152        15.2
2         Mechanical/Structural    132        13.2
3               Sensor/Accuracy    119        11.9
4    Communication/Connectivity     72         7.2
5            Functional Failure     58         5.8
6                          None     42         4.2
7             User/Human Factor     42         4.2
8                   Alarm/Alert     29         2.9
9            Software/Interface     20         2.0
10                      Unknown     14         1.4
11  Environmental/Compatibility     13         1.3
12      Sterility/Contamination     11         1.1
13           Labeling/Packaging      2         0.2


### 1-2. 어떤 제품코드가 어떤 고장 유형이 많이 발생하는가?

In [28]:
def analyze_product_defects(df, top10_product_codes):
    """
    각 제품 코드별 어떤 결함이 많이 발생했는지 확인

    Args : 
        df : 분석할 DF
        top10_product_codes : 제품코드 리스트 
        product_col : 제품코트 컬럼
        defect_col : 고장 유형 컬럼

    Returns : 
    """

    print("=" * 50)
    print("제품 코드별 결함유형")
    print("=" * 50)
    
    # product_code 별로 manufacturer_inspection의 리스트와 count, percentage를 구하는 코드
    product_code_result = (
        df
        .filter(pl.col("device_0_device_report_product_code").is_in(top10_product_codes))
        .group_by(["device_0_device_report_product_code", "defect_type"])
        .agg(pl.len().alias("count"))
        .sort("count", descending=True)
        .group_by("device_0_device_report_product_code")
        .head(3)  # 각 그룹별 상위 3개
        .with_columns(
            (pl.col("count") / pl.col("count").sum().over("device_0_device_report_product_code") * 100)
            .round(2)
            .alias("percentage")
        )
        .sort(["device_0_device_report_product_code", "percentage"], descending=[False, True])
        .collect()
    )

    print(product_code_result)

    return product_code_result

In [29]:
product_code_result = analyze_product_defects(
    df, top10_product_codes
)

제품 코드별 결함유형
shape: (30, 4)
┌─────────────────────────────────┬────────────────────────────┬───────┬────────────┐
│ device_0_device_report_product… ┆ defect_type                ┆ count ┆ percentage │
│ ---                             ┆ ---                        ┆ ---   ┆ ---        │
│ str                             ┆ str                        ┆ u32   ┆ f64        │
╞═════════════════════════════════╪════════════════════════════╪═══════╪════════════╡
│ DTB                             ┆ Sensor/Accuracy            ┆ 18    ┆ 52.94      │
│ DTB                             ┆ Other                      ┆ 13    ┆ 38.24      │
│ DTB                             ┆ Mechanical/Structural      ┆ 3     ┆ 8.82       │
│ FTR                             ┆ Other                      ┆ 29    ┆ 69.05      │
│ FTR                             ┆ Mechanical/Structural      ┆ 9     ┆ 21.43      │
│ …                               ┆ …                          ┆ …     ┆ …          │
│ OZP                      

## 통계

### 1) 카이제곱검정
- **목적**: 특정 제품코드에서 특정 결함이 과대표되는지 검증
- **귀무가설**: 결함 유형과 제품코드는 독립적이다
- **적용**:
    - 관측 빈도 vs 기대 빈도
    - 제품코드 A에서 '배터리 결함' 발생 빈도가 전체 평균 대비 유의하게 높은가?
- **결과 해석**: p < 0.05이면 특정 패턴 존재

### 2) 검정 이후 표준화 잔차 확인

- **목적**: 어떤 셀이 기대치를 크게 벗어나는지 정량화
- **공식**: `(관측값 - 기대값) / sqrt(기대값)`
- **기준**: |z| > 2이면 주목, |z| > 3이면 매우 이례적

### 3) 효과 크기 - Cramer’s V

- **목적**: 통계적 유의성과 별개로 실질적 연관 강도 측정
- **범위**: 0(독립) ~ 1(완전 연관)

In [30]:
def prepare_chi2_data(df, top10_product_codes):
    """
    카이제곱 검정
    - top10 제품만 필터링
    - Unknown, Other 제거
    """

    print("=" * 50)
    print("카이제곱 검정용 데이터")
    print("=" * 50)

    # 상위 10개 제품만 필터링 + Pandas로 변환
    df_filtered = (
        df
        .filter(
            (pl.col("device_0_device_report_product_code").is_in(top10_product_codes))
            & (~pl.col("defect_type").is_in(["Unknown", "Other"])))
        .select(["device_0_device_report_product_code", "defect_type"])
        .collect()
        .to_pandas()  # Pandas로 변환
        )

    print(f"필터링 완료: {len(df_filtered): }행")
    print(f"제품 종류: {df_filtered['device_0_device_report_product_code'].nunique()}개")
    print(f"결함 종류: {df_filtered['defect_type'].nunique()}개")

    return df_filtered

In [31]:
df_filtered = prepare_chi2_data(
    df,
    top10_product_codes
)

카이제곱 검정용 데이터
필터링 완료:  412행
제품 종류: 10개
결함 종류: 11개


In [32]:
chi2_tester = Chi2Test()
chi2_result = chi2_tester.execute(df_filtered, 'device_0_device_report_product_code', 'defect_type')


[기대빈도 확인]
----------------------------------------
최소 기대빈도: 0.08
5 미만 셀: 81/110 (73.6%)
⚠️ 주의: Fisher's exact test 사용 권장
   (기대빈도가 너무 작아 카이제곱 검정 부정확)

[기대빈도]
defect_type                          Alarm/Alert  Communication/Connectivity  \
device_0_device_report_product_code                                            
DTB                                         1.29                        3.67   
FTR                                         0.78                        2.23   
LGW                                         1.98                        5.64   
LWS                                         1.80                        5.11   
MKJ                                         1.01                        2.88   
NVN                                         1.34                        3.80   
OYC                                         1.15                        3.28   
OZD                                         1.11                        3.15   
OZP                                      

defect_type,Alarm/Alert,Communication/Connectivity,Electrical/Power,Environmental/Compatibility,Functional Failure,Labeling/Packaging,Mechanical/Structural,Sensor/Accuracy,Software/Interface,Sterility/Contamination,User/Human Factor
device_0_device_report_product_code,,,,,,,,,,,
DTB,-1.14,-1.39,-1.42,-0.82,-1.54,-0.37,-0.95,5.20,0.05,1.40,-0.14
FTR,0.24,-0.82,-0.02,-0.64,-1.20,-0.29,3.31,-1.31,-0.76,-0.41,0.36
LGW,-1.41,0.57,2.12,-1.02,-0.86,-0.46,-1.40,-0.23,0.45,-0.65,0.92
LWS,-1.34,-2.26,-1.08,-0.97,-0.72,-0.44,-1.19,5.76,-0.28,2.63,-1.27
MKJ,-0.01,-1.70,1.21,-0.73,5.22,2.73,-1.02,-2.11,0.29,-0.46,-0.95
NVN,-1.16,-1.44,-1.86,-0.84,1.62,-0.38,0.71,2.96,-0.99,-0.53,-0.18
OYC,1.72,-1.26,0.43,1.79,-0.08,-0.35,2.51,-2.24,-0.92,-0.49,-1.02
OZD,-1.05,-1.77,-1.97,-0.76,2.77,-0.34,1.70,-1.29,0.20,-0.48,5.03
OZP,2.55,-2.21,2.32,2.47,-0.86,0.38,0.39,-2.70,1.47,-1.17,-0.75


(|잔차| > 2: 유의한 차이, |잔차| > 3: 매우 강한 연관성)

[결론]
✅ p-value(0.0000) < 0.05 → 귀무가설 기각
   device_0_device_report_product_code과(와) defect_type은(는) 관련이 있음   효과 크기: 중간 관계

[사후분석]
   표준화 잔차 |값| > 2인 셀 해석:
   표준화 잔차 |값| > 2인 셀 해석:
   • DTB - Sensor/Accuracy: 예상보다 많음 (잔차=5.20)
   • FTR - Mechanical/Structural: 예상보다 많음 (잔차=3.31)
   • LGW - Electrical/Power: 예상보다 많음 (잔차=2.12)
   • LWS - Communication/Connectivity: 예상보다 적음 (잔차=-2.26)
   • LWS - Sensor/Accuracy: 예상보다 많음 (잔차=5.76)
   • LWS - Sterility/Contamination: 예상보다 많음 (잔차=2.63)
   • MKJ - Functional Failure: 예상보다 많음 (잔차=5.22)
   • MKJ - Labeling/Packaging: 예상보다 많음 (잔차=2.73)
   • MKJ - Sensor/Accuracy: 예상보다 적음 (잔차=-2.11)
   • NVN - Sensor/Accuracy: 예상보다 많음 (잔차=2.96)
   • OYC - Mechanical/Structural: 예상보다 많음 (잔차=2.51)
   • OYC - Sensor/Accuracy: 예상보다 적음 (잔차=-2.24)
   • OZD - Functional Failure: 예상보다 많음 (잔차=2.77)
   • OZD - User/Human Factor: 예상보다 많음 (잔차=5.03)
   • OZP - Alarm/Alert: 예상보다 많음 (잔차=2.55)
   • OZP - Communication/Connectivity: 예상보다 적음 (

### 오즈비
- **목적**: 특정 제품에서 특정 결함 발생 확률 비교
- **공식**: `OR = (제품A 결함발생 오즈) / (다른 제품 결함발생 오즈)`
- **결과 예시**: "인슐린 펌프 X의 배터리 결함 OR = 4.2 (95% CI: 3.1-5.7)"
    - 해석: 다른 제품 대비 4.2배 높은 확률
    - 이후 다시 **카이제곱 검정**으로 특정 제품 대비 전체 제품 오즈비 비교가 유의한지 확인도 가능

In [33]:
import numpy as np
import pandas as pd

def calculate_odds_ratio(
    contingency_table: pd.DataFrame,
    target_product: str,
    target_defect: str,
    chi2_p_value: float = None,
    alpha: float = 0.05
) -> dict:
    """
    분할표에서 특정 제품-결함 조합의 오즈비 계산
    
    Parameters
    ----------
    contingency_table : pd.DataFrame
        pd.crosstab으로 생성한 분할표
        index: 제품 코드, columns: 결함 유형
    target_product : str
        분석 대상 제품 코드
    target_defect : str
        분석 대상 결함 유형
    chi2_p_value : float, optional
        Chi2Test()의 p-value (제공 시 사용)
    alpha : float
        유의수준 (default: 0.05)
    
    Returns
    -------
    dict
        - odds_ratio: 오즈비
        - ci_lower: 95% 신뢰구간 하한
        - ci_upper: 95% 신뢰구간 상한
        - p_value: 카이제곱 검정 p-value
        - interpretation: 해석
        - a, b, c, d: 2x2 분할표 셀 값
    
    Example
    -------
    >>> contingency_table = pd.crosstab(df['product_code'], df['defect_type'])
    >>> chi2_result = chi2_tester.execute(df, 'product_code', 'defect_type')
    >>> result = calculate_odds_ratio(contingency_table, 'INS', '배터리결함', chi2_result.p_value)
    >>> print(f"OR = {result['odds_ratio']:.2f} (95% CI: {result['ci_lower']:.2f}-{result['ci_upper']:.2f})")
    OR = 4.17 (95% CI: 2.52-6.89)
    
    Notes
    -----
    자동으로 2x2 분할표 생성:
                    타겟 결함    기타 결함
    타겟 제품          a           b
    기타 제품          c           d
    """
    
    # a: 타겟 제품 & 타겟 결함
    a = contingency_table.loc[target_product, target_defect] if target_product in contingency_table.index and target_defect in contingency_table.columns else 0
    
    # b: 타겟 제품 & 기타 결함
    b = contingency_table.loc[target_product, :].sum() - a if target_product in contingency_table.index else 0
    
    # c: 기타 제품 & 타겟 결함
    c = contingency_table.loc[:, target_defect].sum() - a if target_defect in contingency_table.columns else 0
    
    # d: 기타 제품 & 기타 결함
    total = contingency_table.values.sum()
    d = total - a - b - c
    
    print(f"\n[2x2 분할표: {target_product} - {target_defect}]")
    print(pd.DataFrame(
        [[a, b], [c, d]],
        index=[f'{target_product}', '기타 제품'],
        columns=[f'{target_defect}', '기타 결함']
    ))
    print()
    
    # 0 처리 (연속성 수정)
    if b == 0 or c == 0 or a == 0 or d == 0:
        a, b, c, d = a + 0.5, b + 0.5, c + 0.5, d + 0.5
    
    # 오즈비 계산
    odds_ratio = (a * d) / (b * c)
    
    # 로그 오즈비의 표준오차 (SE)
    se_log_or = np.sqrt(1/a + 1/b + 1/c + 1/d)
    
    # 95% 신뢰구간 (로그 스케일에서 계산 후 역변환)
    z_critical = 1.96  # 95% CI
    log_or = np.log(odds_ratio)
    ci_lower = np.exp(log_or - z_critical * se_log_or)
    ci_upper = np.exp(log_or + z_critical * se_log_or)
    
    # p-value (Chi2Test 결과 사용)
    p_value = chi2_p_value
    
    # 해석
    if p_value is not None and p_value < alpha:
        if odds_ratio > 1:
            interpretation = f"'{target_product}'에서 '{target_defect}'이(가) 다른 제품 대비 {odds_ratio:.2f}배 높음 (p={p_value:.4f}, 유의함)"
        else:
            interpretation = f"'{target_product}'에서 '{target_defect}'이(가) 다른 제품 대비 {1/odds_ratio:.2f}배 낮음 (p={p_value:.4f}, 유의함)"
    elif p_value is not None:
        interpretation = f"오즈비가 유의하지 않음 (p={p_value:.4f})"
    else:
        interpretation = "p-value 미제공"
    
    return {
        'odds_ratio': odds_ratio,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'p_value': p_value,
        'interpretation': interpretation,
        'a': int(a) if a == int(a) else a,
        'b': int(b) if b == int(b) else b,
        'c': int(c) if c == int(c) else c,
        'd': int(d) if d == int(d) else d
    }

# # 사용 예시
# if __name__ == "__main__":
#     # 예시 분할표 생성
#     data = {
#         '배터리결함': [50, 30, 20],
#         '화상': [100, 80, 60],
#         '오작동': [50, 40, 30]
#     }
#     contingency_table = pd.DataFrame(data, index=['INS', 'PUM', 'DEF'])
    
#     print("전체 분할표:")
#     print(contingency_table)
    
#     # 오즈비 계산
#     result = calculate_odds_ratio(
#         contingency_table=contingency_table,
#         target_product='INS',
#         target_defect='배터리결함',
#         chi2_p_value=0.0001
#     )
    
#     print(f"OR = {result['odds_ratio']:.2f}")
#     print(f"95% CI: ({result['ci_lower']:.2f}, {result['ci_upper']:.2f})")
#     print(f"해석: {result['interpretation']}")

In [34]:
# 1. 분할표 생성
contingency_table = pd.crosstab(
    df_filtered['device_0_device_report_product_code'], 
    df_filtered['defect_type']
)

# 2. 카이제곱 검정 생략: chi2_result 이미 계산됨

# 3. 오즈비 계산 (분할표 + 타겟만 지정)
result = calculate_odds_ratio(
    contingency_table=contingency_table,
    target_product='OZP',      # 분석할 제품 코드
    target_defect='Sensor/Accuracy',  # 분석할 결함 유형
    chi2_p_value=chi2_result.p_value
)

print(f"OR = {result['odds_ratio']:.2f} (95% CI: {result['ci_lower']:.2f}-{result['ci_upper']:.2f})")


[2x2 분할표: OZP - Sensor/Accuracy]
       Sensor/Accuracy  기타 결함
OZP                 14    127
기타 제품               69    202

OR = 0.32 (95% CI: 0.17-0.60)


# 제조사 - 모델 번호 별

## EDA

###  어느 제조사에서 어떤 고장 유형이 많이 발생하는가?

In [35]:
def analyze_manufacturer_defacts(df):
    """
    제조업체 별 결함이 얼마나 발생하는지 분석

    Args : 
        manufacturer_col : 제조사 컬럼
    """
    print("=" * 50)
    print("제조사별 결함유형")
    print("=" * 50)

    # manufacturer별로 manufacturer_inspection의 리스트와 count, percentage를 구하는 코드
    manufacturer_result = (
        df
        .filter(pl.col("device_0_manufacturer_d_name").is_not_null())
        .group_by(["device_0_manufacturer_d_name", "defect_type"])
        .agg(pl.len().alias("count"))
        .sort("count", descending=True)
        .group_by("device_0_manufacturer_d_name")
        .head(5)  # 각 그룹별 상위 5개
        .with_columns(
            (pl.col("count") / pl.col("count").sum().over("device_0_manufacturer_d_name") * 100)
            .round(2)
            .alias("percentage")
        )
        .sort(["device_0_manufacturer_d_name", "percentage"], descending=[False, True])
        .collect()
    )

    print(manufacturer_result)

    return manufacturer_result

In [36]:
manufacturer_result = analyze_manufacturer_defacts(df)


제조사별 결함유형


shape: (209, 4)
┌────────────────────────────────┬───────────────────────┬───────┬────────────┐
│ device_0_manufacturer_d_name   ┆ defect_type           ┆ count ┆ percentage │
│ ---                            ┆ ---                   ┆ ---   ┆ ---        │
│ str                            ┆ str                   ┆ u32   ┆ f64        │
╞════════════════════════════════╪═══════════════════════╪═══════╪════════════╡
│ ABBOTT                         ┆ Sensor/Accuracy       ┆ 4     ┆ 44.44      │
│ ABBOTT                         ┆ Other                 ┆ 2     ┆ 22.22      │
│ ABBOTT                         ┆ Software/Interface    ┆ 1     ┆ 11.11      │
│ ABBOTT                         ┆ Functional Failure    ┆ 1     ┆ 11.11      │
│ ABBOTT                         ┆ Electrical/Power      ┆ 1     ┆ 11.11      │
│ …                              ┆ …                     ┆ …     ┆ …          │
│ ZOLL MANUFACTURING CORPORATION ┆ Mechanical/Structural ┆ 3     ┆ 16.67      │
│ ZOLL MANUFACTURING COR

In [37]:
manufacturer_result.filter((pl.col("count") >= 3)
                           & (~pl.col("defect_type")
                              .is_in(["Unknown", "Other"]))).unique("device_0_manufacturer_d_name")

device_0_manufacturer_d_name,defect_type,count,percentage
str,str,u32,f64
"""ABBOTT""","""Sensor/Accuracy""",4,44.44
"""ABBOTT DIABETES CARE INC""","""Sensor/Accuracy""",5,71.43
"""ABBOTT MEDICAL""","""Electrical/Power""",4,14.29
"""ABBOTT VASCULAR""","""User/Human Factor""",4,44.44
"""ABBOTT VASCULAR INC.""","""User/Human Factor""",5,62.5
…,…,…,…
"""ST. JUDE MEDICAL, INC.(CRM-SYL…","""Sensor/Accuracy""",8,26.67
"""TANDEM DIABETES CARE""","""Electrical/Power""",5,55.56
"""THORATEC CORPORATION""","""Electrical/Power""",4,30.77


# 재활용 기기 / 일회용 기기

In [38]:
# Reuse Flag × Event Type 크로스탭
crosstab = df.group_by(['reprocessed_and_reused_flag', 'event_type']).agg([
    pl.len().alias('count')
]).sort(['reprocessed_and_reused_flag', 'count'], descending=[False, True]).collect()

print("=== Reuse Flag × Event Type ===")
print(crosstab)

# 또는 각 Flag별로 Event Type 분포
print("\n" + "=" * 70)
for flag in ['Y', 'N', None]:
    flag_display = flag if flag else "(NULL)"
    
    flag_total = df.filter(
        pl.col('reprocessed_and_reused_flag') == flag if flag else pl.col('reprocessed_and_reused_flag').is_null()
    ).select(pl.len()).collect().item()
    
    if flag_total == 0:
        continue 
    
    print(f"\n[{flag_display}] 총 {flag_total:,}건")
    print("-" * 70)
    
    event_dist = df.filter(
        pl.col('reprocessed_and_reused_flag') == flag if flag else pl.col('reprocessed_and_reused_flag').is_null()
    ).group_by('event_type').len().sort('len', descending=True).collect()
    
    print(event_dist)

=== Reuse Flag × Event Type ===
shape: (7, 3)
┌─────────────────────────────┬─────────────┬───────┐
│ reprocessed_and_reused_flag ┆ event_type  ┆ count │
│ ---                         ┆ ---         ┆ ---   │
│ str                         ┆ str         ┆ u32   │
╞═════════════════════════════╪═════════════╪═══════╡
│ null                        ┆ Injury      ┆ 16    │
│ null                        ┆ Malfunction ┆ 5     │
│ null                        ┆ Death       ┆ 1     │
│ N                           ┆ Malfunction ┆ 555   │
│ N                           ┆ Injury      ┆ 408   │
│ N                           ┆ Death       ┆ 14    │
│ Y                           ┆ Injury      ┆ 1     │
└─────────────────────────────┴─────────────┴───────┘


[Y] 총 1건
----------------------------------------------------------------------
shape: (1, 2)
┌────────────┬─────┐
│ event_type ┆ len │
│ ---        ┆ --- │
│ str        ┆ u32 │
╞════════════╪═════╡
│ Injury     ┆ 1   │
└────────────┴─────┘

[N] 총 97

### 일회용/재사용 기기의 피해 확인

- 단일

In [39]:
def analyze_single_column_pretty(df, column_name, column_display_name=None):
    """
    단일 컬럼의 분포를 예쁘게 출력하는 함수
    
    Parameters
    ----------
    df : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    column_name : str
        분석할 컬럼명
    column_display_name : str, optional
        출력 시 표시할 컬럼 이름 (None이면 column_name 사용)
    
    Returns
    -------
    polars.DataFrame
        분포 결과 (컬럼값, count, percentage 포함)
    
    Examples
    --------
    >>> # Reprocessed Flag 분포
    >>> analyze_single_column_pretty(df, 'reprocessed_and_reused_flag')
    
    >>> # Event Type 분포
    >>> analyze_single_column_pretty(
    ...     df, 
    ...     'event_type',
    ...     column_display_name='사건 유형'
    ... )
    """
    # 전체 개수
    total = df.select(pl.len()).collect().item()
    
    # 분포 계산
    dist = df.group_by(column_name).agg([
        pl.len().alias('count')
    ]).with_columns([
        (pl.col('count') / total * 100).round(2).alias('percentage')
    ]).sort('count', descending=True).collect()
    
    # 표시 이름
    display_name = column_display_name if column_display_name else column_name
    
    # 출력
    print("=" * 70)
    print(f"{display_name} 분포 (전체: {total:,}건)")
    print("=" * 70)
    print(f"{'값':<20} {'사건 수':>15} {'비율':>15}")
    print("-" * 70)
    
    for row in dist.iter_rows(named=True):
        value = row[column_name] if row[column_name] else "(NULL)"
        count = row['count']
        pct = row['percentage']
        
        # 긴 값은 자르기
        display_value = value[:18] if len(str(value)) > 18 else value
        print(f"{display_value:<20} {count:>15,} {pct:>14.2f}%")
    
    print("=" * 70)
    
    return dist

In [40]:
reuse_dist = analyze_single_column_pretty(
    df, 
    'reprocessed_and_reused_flag',
    column_display_name='재처리/재사용 여부'
)

재처리/재사용 여부 분포 (전체: 1,000건)
값                               사건 수              비율
----------------------------------------------------------------------
N                                977          97.70%
(NULL)                            22           2.20%
Y                                  1           0.10%


- 두 컬럼 크로스탭 분석

In [41]:
def analyze_crosstab(df, column1, column2, 
                     column1_display_name=None, 
                     column2_display_name=None,
                     show_detail=True):
    """
    두 컬럼의 크로스탭(교차 분석)을 수행하는 함수
    
    Parameters
    ----------
    df : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    column1 : str
        첫 번째 컬럼 (예: 'reprocessed_and_reused_flag')
    column2 : str
        두 번째 컬럼 (예: 'event_type')
    column1_display_name : str, optional
        첫 번째 컬럼의 표시 이름
    column2_display_name : str, optional
        두 번째 컬럼의 표시 이름
    show_detail : bool, default=True
        각 column1 값별로 상세 분포를 출력할지 여부
    
    Returns
    -------
    polars.DataFrame
        크로스탭 결과
    
    Examples
    --------
    >>> # Reuse Flag × Event Type
    >>> analyze_crosstab(
    ...     df, 
    ...     'reprocessed_and_reused_flag', 
    ...     'event_type'
    ... )
    
    >>> # 제조사 × Event Type
    >>> analyze_crosstab(
    ...     df,
    ...     'device_0_manufacturer_d_name',
    ...     'event_type',
    ...     column1_display_name='제조사',
    ...     column2_display_name='사건 유형'
    ... )
    
    >>> # 크로스탭만 보기 (상세 분포 제외)
    >>> analyze_crosstab(
    ...     df,
    ...     'reprocessed_and_reused_flag',
    ...     'event_type',
    ...     show_detail=False
    ... )
    """
    # 표시 이름 설정
    col1_name = column1_display_name if column1_display_name else column1
    col2_name = column2_display_name if column2_display_name else column2
    
    # 크로스탭 생성
    crosstab = df.group_by([column1, column2]).agg([
        pl.len().alias('count')
    ]).sort([column1, 'count'], descending=[False, True]).collect()
    
    print("=" * 90)
    print(f"{col1_name} × {col2_name} 크로스탭")
    print("=" * 90)
    print(crosstab)
    
    # 상세 분포 출력
    if show_detail:
        print("\n" + "=" * 90)
        print(f"{col1_name}별 {col2_name} 상세 분포")
        print("=" * 90)
        
        # column1의 고유값 추출 (NULL 포함)
        unique_values = df.select(column1).unique().collect()[column1].to_list()
        
        # NULL도 포함
        if None not in unique_values and df.filter(pl.col(column1).is_null()).select(pl.len()).collect().item() > 0:
            unique_values.append(None)
        
        for value in unique_values:
            value_display = value if value else "(NULL)"
            
            # 해당 값의 총 개수
            value_total = df.filter(
                pl.col(column1) == value if value else pl.col(column1).is_null()
            ).select(pl.len()).collect().item()
            
            if value_total == 0:
                continue
            
            print(f"\n[{value_display}] 총 {value_total:,}건")
            print("-" * 90)
            
            # 해당 값의 column2 분포
            dist = df.filter(
                pl.col(column1) == value if value else pl.col(column1).is_null()
            ).group_by(column2).agg([
                pl.len().alias('count')
            ]).with_columns([
                (pl.col('count') / value_total * 100).round(2).alias('percentage')
            ]).sort('count', descending=True).collect()
            
            print(f"{col2_name:<25} {'건수':>15} {'비율':>15}")
            print("-" * 90)
            
            for row in dist.iter_rows(named=True):
                cat = row[column2] if row[column2] else "(NULL)"
                count = row['count']
                pct = row['percentage']
                
                cat_display = cat[:23] if len(str(cat)) > 23 else cat
                print(f"{cat_display:<25} {count:>15,} {pct:>14.2f}%")
        
        print("=" * 90)
    
    return crosstab

- 재사용 x 이벤트타입

In [42]:
crosstab_result = analyze_crosstab(
    df,
    'reprocessed_and_reused_flag',
    'event_type',
    column1_display_name='재처리 여부',
    column2_display_name='사건 유형'
)

재처리 여부 × 사건 유형 크로스탭
shape: (7, 3)
┌─────────────────────────────┬─────────────┬───────┐
│ reprocessed_and_reused_flag ┆ event_type  ┆ count │
│ ---                         ┆ ---         ┆ ---   │
│ str                         ┆ str         ┆ u32   │
╞═════════════════════════════╪═════════════╪═══════╡
│ null                        ┆ Injury      ┆ 16    │
│ null                        ┆ Malfunction ┆ 5     │
│ null                        ┆ Death       ┆ 1     │
│ N                           ┆ Malfunction ┆ 555   │
│ N                           ┆ Injury      ┆ 408   │
│ N                           ┆ Death       ┆ 14    │
│ Y                           ┆ Injury      ┆ 1     │
└─────────────────────────────┴─────────────┴───────┘

재처리 여부별 사건 유형 상세 분포

[(NULL)] 총 22건
------------------------------------------------------------------------------------------
사건 유형                                  건수              비율
------------------------------------------------------------------------------

# event_type = death 기기 (치명률)

## 치명률

In [43]:
def calculate_cfr_by_device(df, device_column='device_0_generic_name', 
                            event_column='event_type',
                            top_n=None, min_cases=10):
    """
    기기별 치명률(Case Fatality Rate)을 계산하는 함수
    
    치명률(CFR) = (사망 건수 / 해당 기기 총 보고 건수) × 100
    
    Parameters
    ----------
    df : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    device_column : str, default='device_0_generic_name'
        기기 컬럼명
    event_column : str, default='event_type'
        사건 유형 컬럼명
    top_n : int, optional
        상위 N개 기기만 분석 (None이면 전체)
    min_cases : int, default=10
        최소 보고 건수 (이보다 적은 기기는 제외, 통계적 신뢰도 확보)
    
    Returns
    -------
    polars.DataFrame
        기기별 치명률 결과 (기기명, 총건수, 사망건수, 부상건수, 오작동건수, CFR)
    
    Examples
    --------
    >>> # 전체 기기 CFR
    >>> cfr_result = calculate_cfr_by_device(df)
    
    >>> # 상위 20개 기기만
    >>> cfr_top20 = calculate_cfr_by_device(df, top_n=20)
    
    >>> # 최소 100건 이상 보고된 기기만
    >>> cfr_reliable = calculate_cfr_by_device(df, min_cases=100)
    """
    
    # 기기별 전체 건수와 사건 유형별 건수
    device_stats = df.group_by(device_column).agg([
        pl.len().alias('total_cases'),
        (pl.col(event_column) == 'Death').sum().alias('death_count'),
        (pl.col(event_column) == 'Injury').sum().alias('injury_count'),
        (pl.col(event_column) == 'Malfunction').sum().alias('malfunction_count')
    ]).filter(
        pl.col('total_cases') >= min_cases  # 최소 건수 필터
    ).with_columns([
        # CFR 계산
        (pl.col('death_count') / pl.col('total_cases') * 100).round(2).alias('cfr'),
        # 부상률
        (pl.col('injury_count') / pl.col('total_cases') * 100).round(2).alias('injury_rate'),
        # 오작동률
        (pl.col('malfunction_count') / pl.col('total_cases') * 100).round(2).alias('malfunction_rate')
    ]).sort('cfr', descending=True)
    
    # Top N만
    if top_n:
        device_stats = device_stats.head(top_n)
    
    result = device_stats.collect()
    
    # 출력
    print("=" * 120)
    print(f"기기별 치명률(CFR) 분석 (최소 {min_cases}건 이상)")
    print("=" * 120)
    print(f"{'순위':>4} {'기기명':<50} {'총건수':>10} {'사망':>8} {'부상':>8} {'오작동':>10} {'CFR(%)':>10}")
    print("-" * 120)
    
    for i, row in enumerate(result.iter_rows(named=True), 1):
        device = row[device_column] if row[device_column] else "(NULL)"
        total = row['total_cases']
        death = row['death_count']
        injury = row['injury_count']
        mal = row['malfunction_count']
        cfr = row['cfr']
        
        device_short = device[:48] if len(device) > 48 else device
        
        print(f"{i:4d} {device_short:<50} {total:>10,} {death:>8,} {injury:>8,} {mal:>10,} {cfr:>10.2f}%")
    
    print("=" * 120)
    
    # 요약 통계
    print(f"\n◼️ 요약 통계:")
    print(f"  - 분석 기기 수: {len(result):,}개")
    print(f"  - 평균 CFR: {result['cfr'].mean():.2f}%")
    print(f"  - 최대 CFR: {result['cfr'].max():.2f}%")
    print(f"  - 최소 CFR: {result['cfr'].min():.2f}%")
    print(f"  - CFR 중앙값: {result['cfr'].median():.2f}%")
    
    return result

In [44]:
# 최소 10건 이상 보고된 기기의 CFR
cfr_all = calculate_cfr_by_device(df, top_n=20)

기기별 치명률(CFR) 분석 (최소 10건 이상)
  순위 기기명                                                       총건수       사망       부상        오작동     CFR(%)
------------------------------------------------------------------------------------------------------------------------
   1 VENTRICULAR (ASSIST) BYPASS                                11        3        4          4      27.27%
   2 DRUG ELUTING PERMANENT RIGHT VENTRICULAR (RV)              11        1        5          5       9.09%
   3 PULSE GENERATOR, PERMANENT, IMPLANTABLE                    11        1        7          3       9.09%
   4 TEMPORARY NON-ROLLER TYPE LEFT HEART SUPPORT BLO           34        3       17         14       8.82%
   5 DEVICE, HEMOSTASIS, VASCULAR                               12        1        9          2       8.33%
   6 PERMANENT DEFIBRILLATOR ELECTRODES                         11        0        8          3       0.00%
   7 PERMANENT PACEMAKER ELECTRODE                              35        0       15         20 

In [45]:
# 제조사별 치명률
cfr_by_manufacturer = calculate_cfr_by_device(
    df, 
    device_column='device_0_manufacturer_d_name',
    min_cases=50
)

기기별 치명률(CFR) 분석 (최소 50건 이상)
  순위 기기명                                                       총건수       사망       부상        오작동     CFR(%)
------------------------------------------------------------------------------------------------------------------------
   1 MPRI                                                       57        1       26         30       1.75%
   2 BOSTON SCIENTIFIC CORPORATION                              76        1       53         22       1.32%
   3 MEDTRONIC PUERTO RICO OPERATIONS CO.                      302        0       42        260       0.00%
   4 MEDTRONIC MINIMED                                          76        0        1         75       0.00%

◼️ 요약 통계:
  - 분석 기기 수: 4개
  - 평균 CFR: 0.77%
  - 최대 CFR: 1.75%
  - 최소 CFR: 0.00%
  - CFR 중앙값: 0.66%


### CFR 구간별 분포

In [46]:
def analyze_cfr_distribution(df, device_column='device_0_generic_name', min_cases=10):
    """
    CFR 구간별 기기 분포를 분석하는 함수
    
    Parameters
    ----------
    df : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    device_column : str
        기기 컬럼명
    min_cases : int
        최소 보고 건수
    
    Examples
    --------
    >>> analyze_cfr_distribution(df)
    """
    # CFR 계산
    cfr_data = df.group_by(device_column).agg([
        pl.len().alias('total_cases'),
        (pl.col('event_type') == 'Death').sum().alias('death_count')
    ]).filter(
        pl.col('total_cases') >= min_cases
    ).with_columns([
        (pl.col('death_count') / pl.col('total_cases') * 100).alias('cfr')
    ]).collect()
    
    # CFR 구간별 분류
    cfr_ranges = [
        (0, 1, "매우 낮음 (0-1%)"),
        (1, 3, "낮음 (1-3%)"),
        (3, 5, "보통 (3-5%)"),
        (5, 10, "높음 (5-10%)"),
        (10, 100, "매우 높음 (10%+)")
    ]
    
    print("=" * 80)
    print("CFR 구간별 기기 분포")
    print("=" * 80)
    print(f"{'CFR 구간':<25} {'기기 수':>15} {'비율':>15}")
    print("-" * 80)
    
    total_devices = len(cfr_data)
    
    for min_cfr, max_cfr, label in cfr_ranges:
        count = cfr_data.filter(
            (pl.col('cfr') >= min_cfr) & (pl.col('cfr') < max_cfr)
        ).shape[0]
        
        pct = (count / total_devices * 100) if total_devices > 0 else 0
        print(f"{label:<25} {count:>15,} {pct:>14.1f}%")
    
    print("=" * 80)
    print(f"{'총 기기 수':<25} {total_devices:>15,} {100.0:>14.1f}%")
    print("=" * 80)

In [47]:
# CFR 구간별 기기 분포
analyze_cfr_distribution(df, min_cases=50)

CFR 구간별 기기 분포
CFR 구간                               기기 수              비율
--------------------------------------------------------------------------------
매우 낮음 (0-1%)                            1          100.0%
낮음 (1-3%)                               0            0.0%
보통 (3-5%)                               0            0.0%
높음 (5-10%)                              0            0.0%
매우 높음 (10%+)                            0            0.0%
총 기기 수                                  1          100.0%


In [48]:
def analyze_device_detail(df, device_name, device_column='device_0_generic_name'):
    """
    특정 기기의 상세 통계 분석
    
    Parameters
    ----------
    df : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    device_name : str
        분석할 기기명
    device_column : str
        기기 컬럼명
    
    Examples
    --------
    >>> analyze_device_detail(df, 'Catheter, Intravascular, Therapeutic')
    """
    device_data = df.filter(pl.col(device_column) == device_name).collect()
    
    total = len(device_data)
    death = device_data.filter(pl.col('event_type') == 'Death').shape[0]
    injury = device_data.filter(pl.col('event_type') == 'Injury').shape[0]
    mal = device_data.filter(pl.col('event_type') == 'Malfunction').shape[0]
    
    cfr = (death / total * 100) if total > 0 else 0
    injury_rate = (injury / total * 100) if total > 0 else 0
    mal_rate = (mal / total * 100) if total > 0 else 0
    
    print("=" * 80)
    print(f"기기 상세 분석: {device_name}")
    print("=" * 80)
    print(f"\n◼️ 기초 통계:")
    print(f"  총 보고 건수: {total:,}건")
    print(f"\n◼️ 사건 유형별 분포:")
    print(f"  • 사망 (Death):        {death:>8,}건 ({cfr:>5.2f}%) ⚠️ CFR")
    print(f"  • 부상 (Injury):       {injury:>8,}건 ({injury_rate:>5.2f}%)")
    print(f"  • 오작동 (Malfunction): {mal:>8,}건 ({mal_rate:>5.2f}%)")
    print("=" * 80)

In [49]:
# 특정 기기 상세 분석
analyze_device_detail(df,'IMPLANTABLE CARDIOVERTER DEFIBRILLATOR (NON-CRT)')

기기 상세 분석: IMPLANTABLE CARDIOVERTER DEFIBRILLATOR (NON-CRT)

◼️ 기초 통계:
  총 보고 건수: 26건

◼️ 사건 유형별 분포:
  • 사망 (Death):               0건 ( 0.00%) ⚠️ CFR
  • 부상 (Injury):             14건 (53.85%)
  • 오작동 (Malfunction):       12건 (46.15%)



**기기 결함 종류 분포**

**결함 별 환자 피해/사건**

**기기 결함별 부품 분포**

**종합결함 확인표**

# device_class (EDA)
* 환자 피해 patient_harm
* 문제 기기 problem_components
* 당시 상황 incident_summary
* 결함 여부 defect_confirmed
* 결함 종류 defect_type
* 기기 실험 요약 inspection_actions

### 이상기기 top_n

In [50]:
def get_top_n_by_column(df, group_column, top_n=10, column_display_name=None, verbose=True):
    """
    특정 컬럼을 기준으로 상위 N개 항목을 추출하고 비율과 함께 출력하는 함수
    
    Parameters
    ----------
    df : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    group_column : str
        그룹화할 컬럼명 (예: 'device_0_generic_name')
    top_n : int, default=10
        상위 N개 항목 추출
    column_display_name : str, optional
        출력 시 표시할 컬럼 이름 (None이면 group_column 사용)
    verbose : bool, default=True
        결과를 출력할지 여부
    
    Returns
    -------
    polars.DataFrame
        상위 N개 항목 (컬럼명, count, percentage 포함)
    
    Examples
    --------
    >>> # 기본 사용 - 상위 20개 기기
    >>> top_devices = get_top_n_by_column(df, 'device_0_generic_name', top_n=20)
    
    >>> # 제조사 상위 10개
    >>> top_manufacturers = get_top_n_by_column(
    ...     df, 
    ...     'device_0_manufacturer_d_name', 
    ...     top_n=10,
    ...     column_display_name='제조사'
    ... )
    
    >>> # 출력 없이 결과만
    >>> result = get_top_n_by_column(df, 'device_0_generic_name', top_n=5, verbose=False)
    """
    # 전체 개수
    total = df.select(pl.len()).collect().item()
    
    # 상위 N개 추출
    top_items = df.group_by(group_column).agg([
        pl.len().alias('count')
    ]).with_columns([
        (pl.col('count') / total * 100).round(2).alias('percentage')
    ]).sort('count', descending=True).head(top_n).collect()
    
    # 출력
    if verbose:
        display_name = column_display_name if column_display_name else group_column
        
        print("=" * 90)
        print(f"{display_name} 사건 수 Top {top_n} (전체: {total:,}건)")
        print("=" * 90)
        print()
        
        for i, row in enumerate(top_items.iter_rows(named=True), 1):
            value = row[group_column] if row[group_column] else "(NULL)"
            count = row['count']
            pct = row['percentage']
            
            # 긴 값은 70자로 자르기
            display_value = value[:70] if len(value) > 70 else value
            
            print(f"{i:2d}. {display_value:70s} {count:>8,}건 ({pct:>5.1f}%)")
    
    return top_items

In [51]:
top_devices = get_top_n_by_column(
    df, 
    'device_0_generic_name', 
    top_n=20
)

device_0_generic_name 사건 수 Top 20 (전체: 1,000건)

 1. AUTOMATED INSULIN DOSING DEVICE SYSTEM, SINGLE HORMONAL CONTROL             228건 ( 22.8%)
 2. PROSTHESIS, BREAST, NONINFLATABLE, INTERNAL, SILICONE GEL-FILLED             46건 (  4.6%)
 3. PERMANENT PACEMAKER ELECTRODE                                                35건 (  3.5%)
 4. SENSOR, GLUCOSE, INVASIVE , NON-ADJUNCTIVE                                   34건 (  3.4%)
 5. TEMPORARY NON-ROLLER TYPE LEFT HEART SUPPORT BLOOD PUMP                      34건 (  3.4%)
 6. PUMP, INFUSION, INSULIN, TO BE USED WITH INVASIVE GLUCOSE SENSOR             33건 (  3.3%)
 7. STIMULATOR, SPINAL-CORD, TOTALLY IMPLANTED FOR                               28건 (  2.8%)
 8. IMPLANTABLE CARDIOVERTER DEFIBRILLATOR (NON-CRT)                             26건 (  2.6%)
 9. AUTOMATED INSULIN DOSING, THRESHOLD SUSPEND                                  21건 (  2.1%)
10. WEARABLE CARDIOVERTER DEFIBRILLATOR                                          21건 (  2.1%)
11. PROSTHES

### device 3 기기 event_type 분포 확인

In [52]:
def analyze_top_n_by_category(df, group_column, category_column, top_n=3, 
                               group_display_name=None, category_display_name=None, 
                               verbose=True):
    """
    상위 N개 항목에 대해 카테고리별 분포를 분석하는 함수
    
    Parameters
    ----------
    df : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    group_column : str
        그룹화할 주요 컬럼 (예: 'device_0_generic_name')
    category_column : str
        분포를 확인할 카테고리 컬럼 (예: 'event_type')
    top_n : int, default=3
        상위 N개 항목 분석
    group_display_name : str, optional
        그룹 컬럼의 표시 이름 (None이면 group_column 사용)
    category_display_name : str, optional
        카테고리 컬럼의 표시 이름 (None이면 category_column 사용)
    verbose : bool, default=True
        결과를 출력할지 여부
    
    Returns
    -------
    dict
        각 상위 항목별 카테고리 분포 딕셔너리
    
    Examples
    --------
    >>> # Top 3 기기별 Event Type 분포
    >>> result = analyze_top_n_by_category(
    ...     df, 
    ...     'device_0_generic_name', 
    ...     'event_type',
    ...     top_n=3
    ... )
    
    >>> # Top 5 제조사별 Event Type 분포
    >>> result = analyze_top_n_by_category(
    ...     df,
    ...     'device_0_manufacturer_d_name',
    ...     'event_type',
    ...     top_n=5,
    ...     group_display_name='제조사',
    ...     category_display_name='사건 유형'
    ... )
    
    >>> # Top 10 기기별 재처리 여부 분포
    >>> result = analyze_top_n_by_category(
    ...     df,
    ...     'device_0_generic_name',
    ...     'reprocessed_and_reused_flag',
    ...     top_n=10,
    ...     category_display_name='재처리 여부'
    ... )
    """
    # 상위 N개 항목 추출
    top_items = df.group_by(group_column).len()\
        .sort('len', descending=True).head(top_n).collect()
    
    # 결과 저장용 딕셔너리
    results = {}
    
    # 표시 이름 설정
    group_name = group_display_name if group_display_name else group_column
    category_name = category_display_name if category_display_name else category_column
    
    if verbose:
        print("=" * 90)
        print(f"Top {top_n} {group_name}별 {category_name} 분포")
        print("=" * 90)
    
    # 각 항목에 대해 카테고리 분포 계산
    for rank, row in enumerate(top_items.iter_rows(named=True), 1):
        item_value = row[group_column]
        total_count = row['len']
        
        # 해당 항목의 카테고리 분포
        dist = df.filter(
            pl.col(group_column) == item_value
        ).group_by(category_column).agg([
            pl.len().alias('count')
        ]).with_columns([
            (pl.col('count') / total_count * 100).round(2).alias('percentage')
        ]).sort('count', descending=True).collect()
        
        # 결과 저장
        results[item_value] = dist
        
        # 출력
        if verbose:
            print(f"\n[{rank}위] {item_value}")
            print(f"총 사건 수: {total_count:,}건")
            print("-" * 90)
            print(f"{category_name:<20} {'건수':>15} {'비율':>15}")
            print("-" * 90)
            
            for cat_row in dist.iter_rows(named=True):
                category = cat_row[category_column] if cat_row[category_column] else "(NULL)"
                count = cat_row['count']
                pct = cat_row['percentage']
                print(f"{category:<20} {count:>15,} {pct:>14.1f}%")
    
    if verbose:
        print("=" * 90)
    
    return results

In [53]:
result = analyze_top_n_by_category(
    df,
    'device_0_generic_name',
    'event_type',
    top_n=3
)

Top 3 device_0_generic_name별 event_type 분포

[1위] AUTOMATED INSULIN DOSING DEVICE SYSTEM, SINGLE HORMONAL CONTROL
총 사건 수: 228건
------------------------------------------------------------------------------------------
event_type                        건수              비율
------------------------------------------------------------------------------------------
Malfunction                      200           87.7%
Injury                            28           12.3%

[2위] PROSTHESIS, BREAST, NONINFLATABLE, INTERNAL, SILICONE GEL-FILLED
총 사건 수: 46건
------------------------------------------------------------------------------------------
event_type                        건수              비율
------------------------------------------------------------------------------------------
Injury                            46          100.0%

[3위] PERMANENT PACEMAKER ELECTRODE
총 사건 수: 35건
------------------------------------------------------------------------------------------
event_type             

### 결함 종류별 문제 기기 부품 분석

In [54]:
def analyze_defect_components(df, defect_type, top_n=10):
    """
    특정 결함 종류의 문제 기기 부품 분석
    
    Parameters
    ----------
    df : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    defect_type : str
        분석할 결함 종류
    top_n : int, default=10
        상위 N개 문제 부품 표시
    
    Returns
    -------
    polars.DataFrame
        문제 부품 분포
    
    Examples
    --------
    >>> # 특정 결함의 문제 부품
    >>> analyze_defect_components(df, 'Software Failure', top_n=15)
    """
    # 해당 결함 필터링
    defect_data = df.filter(
        pl.col('defect_type') == defect_type
    )
    
    total = defect_data.select(pl.len()).collect().item()
    
    # 문제 부품 분포
    component_dist = defect_data.group_by('problem_components').agg([
        pl.len().alias('count')
    ]).with_columns([
        (pl.col('count') / total * 100).round(2).alias('percentage')
    ]).sort('count', descending=True).head(top_n).collect()
    
    print("=" * 100)
    print(f"결함 종류: {defect_type}")
    print(f"문제 기기 부품 분석 (전체: {total:,}건)")
    print("=" * 100)
    print(f"{'순위':>4} {'문제 부품':<60} {'건수':>12} {'비율':>12}")
    print("-" * 100)
    
    for i, row in enumerate(component_dist.iter_rows(named=True), 1):
        component = row['problem_components']
        count = row['count']
        pct = row['percentage']
        
        component_display = component[:58] if component and len(component) > 58 else (component or "(NULL)")
        print(f"{i:4d} {component_display:<60} {count:>12,} {pct:>11.2f}%")
    
    print("=" * 100)
    
    return component_dist

In [55]:
# "Sensor/Accuracy" 결함의 문제 부품
components = analyze_defect_components(df, 'Sensor/Accuracy', top_n=15)

결함 종류: Sensor/Accuracy
문제 기기 부품 분석 (전체: 119건)
  순위 문제 부품                                                                  건수           비율
----------------------------------------------------------------------------------------------------
   1 ['sensor']                                                             16       13.45%
   2 ['lead', 'sensor']                                                     12       10.08%
   3 ['lead', 'impedance']                                                  10        8.40%
   4 []                                                                      5        4.20%
   5 ['lead', 'sensing']                                                     5        4.20%
   6 ['lead', 'device']                                                      4        3.36%
   7 ['sensor', 'software']                                                  2        1.68%
   8 ['impedance']                                                           2        1.68%
   9 ['lead']            

### 분포도 확인 함수 코드

In [56]:
def analyze_column_distribution(df, column_name, top_n=None, show_null=True, verbose=True):
    """
    특정 컬럼의 값 분포와 비율을 분석하는 함수
    
    Parameters
    ----------
    df : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    column_name : str
        분석할 컬럼명
    top_n : int, optional
        상위 N개만 표시 (None이면 전체 표시)
    show_null : bool, default=True
        NULL 값 개수를 별도로 표시할지 여부
    verbose : bool, default=True
        결과를 출력할지 여부
    
    Returns
    -------
    polars.DataFrame
        분포 결과 (컬럼명, count, percentage 포함)
    
    Examples
    --------
    >>> # 기본 사용
    >>> result = analyze_column_distribution(df, 'event_type')
    
    >>> # 상위 10개만
    >>> result = analyze_column_distribution(df, 'device_0_generic_name', top_n=10)
    
    >>> # 출력 없이 결과만
    >>> result = analyze_column_distribution(df, 'event_type', verbose=False)
    """
    # 전체 개수
    total = df.select(pl.len()).collect().item()
    
    # 분포 계산
    dist = df.group_by(column_name).agg([
        pl.len().alias('count')
    ]).with_columns([
        (pl.col('count') / total * 100).round(2).alias('percentage')
    ]).sort('count', descending=True)
    
    # 상위 N개만
    if top_n is not None:
        dist = dist.head(top_n)
    
    result = dist
    
    # NULL 개수 확인
    null_count = df.filter(pl.col(column_name).is_null()).select(pl.len()).collect().item()
    
    # 출력
    if verbose:
        print("=" * 80)
        print(f"{column_name} 분포 (전체: {total:,}건)")
        if top_n:
            print(f"(상위 {top_n}개만 표시)")
        print("=" * 80)
        print(result)
        
        if show_null:
            print(f"\nNULL 값: {null_count:,}개 ({null_count/total*100:.2f}%)")
    
    return result

In [57]:
# event_type 분포 확인
event_dist = analyze_column_distribution(df, 'event_type')
print(event_dist)

event_type 분포 (전체: 1,000건)
naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

SORT BY [descending: [true]] [col("count")]
   WITH_COLUMNS:
   [[([(col("count")) / (1000)]) * (100.0)].round().alias("percentage")] 
    AGGREGATE[maintain_order: false]
      [len().alias("count")] BY [col("event_type")]
      FROM
      DF ["adverse_event_flag", "date_added", "date_changed", "date_facility_aware", ...]; PROJECT */409 COLUMNS

NULL 값: 0개 (0.00%)
naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

SORT BY [descending: [true]] [col("count")]
   WITH_COLUMNS:
   [[([(col("count")) / (1000)]) * (100.0)].round().alias("percentage")] 
    AGGREGATE[maintain_order: false]
      [len().alias("count")] BY [col("event_type")]
      FROM
      DF ["adverse_event_flag", "date_added", "date_changed", "date_facility_aware", ...]; PROJECT */409 COLUMNS


### 기기 결함 종류 분포 분석

In [58]:
def analyze_defect_types(df, top_n=20, min_cases=5):
    """
    기기 결함 종류 분포를 분석하는 함수
    
    Parameters
    ----------
    df : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    top_n : int, default=20
        상위 N개 결함 종류 표시
    min_cases : int, default=5
        최소 발생 건수 (이보다 적은 결함은 제외)
    
    Returns
    -------
    polars.DataFrame
        결함 종류별 분포 결과
    
    Examples
    --------
    >>> # 전체 결함 종류 분포
    >>> defect_dist = analyze_defect_types(df)
    
    >>> # 상위 30개만
    >>> defect_top30 = analyze_defect_types(df, top_n=30)
    """
    # 전체 개수
    total = df.select(pl.len()).collect().item()
    
    # 결함 종류 분포
    defect_dist = df.group_by('defect_type').agg([
        pl.len().alias('count')
    ]).filter(
        pl.col('count') >= min_cases
    ).with_columns([
        (pl.col('count') / total * 100).round(2).alias('percentage')
    ]).sort('count', descending=True).head(top_n).collect()
    
    # 결함 확인 여부
    defect_confirmed = df.group_by('defect_confirmed').len().collect()
    
    print("=" * 100)
    print(f"기기 결함 종류 분석 (전체: {total:,}건)")
    print("=" * 100)
    
    # 결함 확인 여부 먼저 출력
    print("\n◼️ 결함 확인 여부:")
    print("-" * 100)
    for row in defect_confirmed.iter_rows(named=True):
        confirmed = row['defect_confirmed']
        count = row['len']
        pct = (count / total * 100)
        confirmed_display = confirmed if confirmed else "(NULL)"
        print(f"  {confirmed_display:<20} {count:>10,}건 ({pct:>5.2f}%)")
    
    # 결함 종류 분포
    print(f"\n◼️ 결함 종류 Top {top_n} (최소 {min_cases}건 이상):")
    print("-" * 100)
    print(f"{'순위':>4} {'결함 종류':<60} {'건수':>12} {'비율':>12}")
    print("-" * 100)
    
    for i, row in enumerate(defect_dist.iter_rows(named=True), 1):
        defect_type = row['defect_type']
        count = row['count']
        pct = row['percentage']
        
        defect_display = defect_type[:58] if defect_type and len(defect_type) > 58 else (defect_type or "(NULL)")
        print(f"{i:4d} {defect_display:<60} {count:>12,} {pct:>11.2f}%")
    
    print("=" * 100)
    
    return defect_dist

In [59]:
# 결함 종류 Top 20
defect_distribution = analyze_defect_types(df, top_n=12)

기기 결함 종류 분석 (전체: 1,000건)

◼️ 결함 확인 여부:
----------------------------------------------------------------------------------------------------
  (NULL)                       30건 ( 3.00%)
  False                       593건 (59.30%)
  True                        377건 (37.70%)

◼️ 결함 종류 Top 12 (최소 5건 이상):
----------------------------------------------------------------------------------------------------
  순위 결함 종류                                                                  건수           비율
----------------------------------------------------------------------------------------------------
   1 Other                                                                 294       29.40%
   2 Electrical/Power                                                      152       15.20%
   3 Mechanical/Structural                                                 132       13.20%
   4 Sensor/Accuracy                                                       119       11.90%
   5 Communication/Connectivity      

### 결함 종류별 환자 피해/사건 유형 분석

In [60]:
def analyze_defect_impact(df, top_defects=10):
    """
    주요 결함 종류별 환자 피해 및 사건 유형 분석
    
    Parameters
    ----------
    df : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    top_defects : int, default=10
        분석할 상위 결함 종류 개수
    
    Returns
    -------
    dict
        결함 종류별 분석 결과
    
    Examples
    --------
    >>> # 상위 10개 결함의 영향 분석
    >>> impact_result = analyze_defect_impact(df, top_defects=10)
    
    >>> # 상위 5개만
    >>> impact_top5 = analyze_defect_impact(df, top_defects=5)
    """
    # 상위 결함 종류 추출
    top_defects_list = df.group_by('defect_type').len()\
        .sort('len', descending=True).head(top_defects).collect()
    
    results = {}
    
    print("=" * 110)
    print(f"상위 {top_defects}개 결함 종류별 영향 분석")
    print("=" * 110)
    
    for rank, row in enumerate(top_defects_list.iter_rows(named=True), 1):
        defect_type = row['defect_type']
        total_count = row['len']
        
        if not defect_type:
            defect_type = "(NULL)"
        
        # 해당 결함의 데이터 필터링
        defect_data = df.filter(
            pl.col('defect_type') == defect_type if defect_type != "(NULL)" 
            else pl.col('defect_type').is_null()
        )
        
        # Event Type 분포
        event_dist = defect_data.group_by('event_type').agg([
            pl.len().alias('count')
        ]).with_columns([
            (pl.col('count') / total_count * 100).round(2).alias('percentage')
        ]).sort('count', descending=True).collect()
        
        # 환자 피해 여부 (NULL이 아닌 경우)
        harm_count = defect_data.filter(
            pl.col('patient_harm').is_not_null()
        ).select(pl.len()).collect().item()
        
        # CFR 계산
        death_count = defect_data.filter(pl.col('event_type') == 'Death').select(pl.len()).collect().item()
        cfr = (death_count / total_count * 100) if total_count > 0 else 0
        
        results[defect_type] = {
            'total': total_count,
            'event_dist': event_dist,
            'harm_count': harm_count,
            'cfr': cfr
        }
        
        # 출력
        print(f"\n[{rank}위] {defect_type}")
        print(f"총 {total_count:,}건 | 환자 피해 기록: {harm_count:,}건 | CFR: {cfr:.2f}%")
        print("-" * 110)
        print(f"{'사건 유형':<20} {'건수':>15} {'비율':>15}")
        print("-" * 110)
        
        for event_row in event_dist.iter_rows(named=True):
            event = event_row['event_type'] if event_row['event_type'] else "(NULL)"
            count = event_row['count']
            pct = event_row['percentage']
            print(f"{event:<20} {count:>15,} {pct:>14.2f}%")
    
    print("=" * 110)
    
    return results

In [61]:
# 상위 10개 결함의 환자 피해/사건 유형
impact_analysis = analyze_defect_impact(df, top_defects=10)

상위 10개 결함 종류별 영향 분석

[1위] Other
총 294건 | 환자 피해 기록: 294건 | CFR: 2.72%
--------------------------------------------------------------------------------------------------------------
사건 유형                             건수              비율
--------------------------------------------------------------------------------------------------------------
Injury                           190          64.63%
Malfunction                       96          32.65%
Death                              8           2.72%

[2위] Electrical/Power
총 152건 | 환자 피해 기록: 152건 | CFR: 1.32%
--------------------------------------------------------------------------------------------------------------
사건 유형                             건수              비율
--------------------------------------------------------------------------------------------------------------
Malfunction                      106          69.74%
Injury                            44          28.95%
Death                              2           1.32%

[3

### 종합 결함 분석 대시보드

In [62]:
def comprehensive_defect_analysis(df):
    """
    결함에 대한 종합적인 분석을 수행하는 함수
    
    Parameters
    ----------
    df : polars.LazyFrame or polars.DataFrame
        분석할 데이터프레임
    
    Examples
    --------
    >>> comprehensive_defect_analysis(df)
    """
    total = df.select(pl.len()).collect().item()
    
    print("=" * 120)
    print("◼️ Class 3 기기 결함 종합 분석 대시보드")
    print("=" * 120)
    
    # 1. 결함 확인 여부
    print("\n◼️ 1. 제조사 검사 - 결함 확인 여부")
    print("-" * 120)
    defect_confirmed = df.group_by('defect_confirmed').agg([
        pl.len().alias('count')
    ]).with_columns([
        (pl.col('count') / total * 100).round(2).alias('percentage')
    ]).sort('count', descending=True).collect()
    
    for row in defect_confirmed.iter_rows(named=True):
        confirmed = row['defect_confirmed'] or "(NULL)"
        count = row['count']
        pct = row['percentage']
        print(f"  {confirmed:<30} {count:>15,}건 ({pct:>5.2f}%)")
    
    # 2. 결함 종류 Top 10
    print("\n◼️ 2. 결함 종류 Top 10")
    print("-" * 120)
    defect_types = df.filter(
        pl.col('defect_type').is_not_null()
    ).group_by('defect_type').agg([
        pl.len().alias('count')
    ]).with_columns([
        (pl.col('count') / total * 100).round(2).alias('percentage')
    ]).sort('count', descending=True).head(10).collect()
    
    for i, row in enumerate(defect_types.iter_rows(named=True), 1):
        defect = row['defect_type'][:50]
        count = row['count']
        pct = row['percentage']
        print(f"  {i:2d}. {defect:<50} {count:>10,}건 ({pct:>5.2f}%)")
    
    # 3. 문제 부품 Top 10
    print("\n◼️ 3. 문제 기기 부품 Top 10")
    print("-" * 120)
    components = df.filter(
        pl.col('problem_components').is_not_null()
    ).group_by('problem_components').agg([
        pl.len().alias('count')
    ]).sort('count', descending=True).head(10).collect()
    
    for i, row in enumerate(components.iter_rows(named=True), 1):
        component = row['problem_components'][:50]
        count = row['count']
        print(f"  {i:2d}. {component:<50} {count:>10,}건")
    
    # 4. 환자 피해 요약
    print("\n◼️  4. 환자 피해 기록")
    print("-" * 120)
    harm_count = df.filter(
        pl.col('patient_harm').is_not_null()
    ).select(pl.len()).collect().item()
    harm_pct = (harm_count / total * 100)
    
    print(f"  환자 피해 기록 있음: {harm_count:>10,}건 ({harm_pct:>5.2f}%)")
    print(f"  환자 피해 기록 없음: {total - harm_count:>10,}건 ({100 - harm_pct:>5.2f}%)")
    
    # 5. 검사 조치 요약
    print("\n◼️ 5. 제조사 검사 조치")
    print("-" * 120)
    inspection_count = df.filter(
        pl.col('inspection_actions').is_not_null()
    ).select(pl.len()).collect().item()
    inspection_pct = (inspection_count / total * 100)
    
    print(f"  검사 조치 기록 있음: {inspection_count:>10,}건 ({inspection_pct:>5.2f}%)")
    print(f"  검사 조치 기록 없음: {total - inspection_count:>10,}건 ({100 - inspection_pct:>5.2f}%)")
    
    print("\n" + "=" * 120)

In [63]:
# 전체 결함 분석 한눈에 보기
comprehensive_defect_analysis(df)

◼️ Class 3 기기 결함 종합 분석 대시보드

◼️ 1. 제조사 검사 - 결함 확인 여부
------------------------------------------------------------------------------------------------------------------------
  False                                      593건 (59.30%)
  True                                       377건 (37.70%)
  (NULL)                                      30건 ( 3.00%)

◼️ 2. 결함 종류 Top 10
------------------------------------------------------------------------------------------------------------------------
   1. Other                                                     294건 (29.40%)
   2. Electrical/Power                                          152건 (15.20%)
   3. Mechanical/Structural                                     132건 (13.20%)
   4. Sensor/Accuracy                                           119건 (11.90%)
   5. Communication/Connectivity                                 72건 ( 7.20%)
   6. Functional Failure                                         58건 ( 5.80%)
   7. User/Human Factor                 